In [1]:
import numpy as np

In [2]:
np.set_printoptions(suppress=True, linewidth=300,precision=2)

In [3]:
data0=np.genfromtxt("loan-data.csv",delimiter=";", skip_header=1, autostrip=True)
print(data0)

[[48010226.           nan    35000.   ...         nan         nan     9452.96]
 [57693261.           nan    30000.   ...         nan         nan     4679.7 ]
 [59432726.           nan    15000.   ...         nan         nan     1969.83]
 ...
 [50415990.           nan    10000.   ...         nan         nan     2185.64]
 [46154151.           nan         nan ...         nan         nan     3199.4 ]
 [66055249.           nan    10000.   ...         nan         nan      301.9 ]]


Separ tabelul in 2:unul cu stringuri (string_cols) si unul cu numere (numeric_cols)

In [4]:
numeric_cols_indices=np.where(np.isnan(np.nanmean(data0, axis=0))==False)[0]
string_cols_indices=np.where(np.isnan(np.nanmean(data0, axis=0)))[0]
print(numeric_cols_indices)
print(string_cols_indices)

[ 0  2  4  6  7 13]
[ 1  3  5  8  9 10 11 12]


C:\Users\EMELNIAP8\AppData\Local\Temp\ipykernel_23368\4260221152.py:1: RuntimeWarning: Mean of empty slice
  numeric_cols_indices=np.where(np.isnan(np.nanmean(data0, axis=0))==False)[0]
C:\Users\EMELNIAP8\AppData\Local\Temp\ipykernel_23368\4260221152.py:2: RuntimeWarning: Mean of empty slice
  string_cols_indices=np.where(np.isnan(np.nanmean(data0, axis=0)))[0]


In [5]:
numeric_cols=np.genfromtxt("loan-data.csv",delimiter=";", skip_header=1, usecols=numeric_cols_indices,autostrip=True)
string_cols=np.genfromtxt("loan-data.csv",delimiter=";", skip_header=1, usecols=string_cols_indices, dtype=np.str_,autostrip=True)
print(numeric_cols, end="\n\n")
print(string_cols)

[[48010226.      35000.      35000.         13.33     1184.86     9452.96]
 [57693261.      30000.      30000.           nan      938.57     4679.7 ]
 [59432726.      15000.      15000.           nan      494.86     1969.83]
 ...
 [50415990.      10000.      10000.           nan         nan     2185.64]
 [46154151.           nan    10000.         16.55      354.3      3199.4 ]
 [66055249.      10000.      10000.           nan      309.97      301.9 ]]

[['May-15' 'Current' '36 months' ... 'Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226' 'CA']
 ['' 'Current' '36 months' ... 'Source Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261' 'NY']
 ['Sep-15' 'Current' '36 months' ... 'Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726' 'PA']
 ...
 ['Jun-15' 'Current' '36 months' ... 'Source Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990' 'CA']
 ['Apr-15' 'Current' '36 m

In [6]:
numeric_header=np.genfromtxt("loan-data.csv",delimiter=";", skip_footer=numeric_cols.shape[0], usecols=numeric_cols_indices,autostrip=True, dtype=np.str_)
string_header=np.genfromtxt("loan-data.csv",delimiter=";", skip_footer=string_cols.shape[0], usecols=string_cols_indices,autostrip=True, dtype=np.str_)
print(numeric_header)
print(string_header)

['id' 'loan_amount' 'funded_amount' 'int_rate' 'installment' 'total_payment']
['issue_date' 'loan_status' 'term' 'grade' 'sub_grade' 'verification_status' 'url' 'addr_state']


"Issue Date"

In [7]:
print(np.unique(string_cols[:,0]))

['' 'Apr-15' 'Aug-15' 'Dec-15' 'Feb-15' 'Jan-15' 'Jul-15' 'Jun-15' 'Mar-15' 'May-15' 'Nov-15' 'Oct-15' 'Sep-15']


Elimin '-15'din coloana 'issue-date' de pe toate liniile, pentru ca are aceeasi valoare pe toate liniile

In [8]:
string_cols[:,0] = np.char.replace(string_cols[:,0],  '-15',  '')
print(np.unique(string_cols[:,0]))

['' 'Apr' 'Aug' 'Dec' 'Feb' 'Jan' 'Jul' 'Jun' 'Mar' 'May' 'Nov' 'Oct' 'Sep']


Inlocuiesc fiecare luna cu echivalentul ei numeric ('Jan'->'1')

In [9]:
months=np.array(["", "Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct", "Nov", "Dec"])

In [10]:
for i in range(months.shape[0]):
    string_cols[:,0]=np.where(string_cols[:,0]==months[i], str(i), string_cols[:,0])
    
print(np.unique(string_cols[:,0]))

['0' '1' '10' '11' '12' '2' '3' '4' '5' '6' '7' '8' '9']


In [11]:
string_header[0]="issue_month"

"Loan Status"

In [12]:
print(np.unique(string_cols[:,1]))
print(string_cols[:,1])

['' 'Charged Off' 'Current' 'Default' 'Fully Paid' 'In Grace Period' 'Issued' 'Late (16-30 days)' 'Late (31-120 days)']
['Current' 'Current' 'Current' ... 'Current' 'Current' 'Current']


"Charged Off" = debitorul nu a platit o perioada lunga si banca considera creditul pierdere, inchide oficial creditul in contabilitate
"Current" = imprumutul este la zi, toate platile au fost facute conform graficului
"Default" = debitorul este in incapacitate de plata
"Fully Paid" = creditul a fost platit integral
"In Grace Period" = debitorul a intarziat plata, dar se afla inca in perioada de gratie
"Issued" = creditul a fost acordat recent, banii au fost transferati catre debitor
"Late" = plata intarziata

Pentru a determina daca un candidat e stabil financiar si eligibil pentru credit, putem foosi doar 2 valori:0 si 1. ["charged off", "default", "late (31-120 days)", no data]->0, iar ["current", "fully paid", "in grace period", "issued", "late(16-30 days)"]->1

In [13]:
good_status=np.array(["Current", "Fully Paid", "In Grace Period", "Issued", "Late (16-30 days)"])
string_cols[:,1]=np.where(np.isin(string_cols[:,1], good_status), "1" , "0")


In [14]:
print(np.unique(string_cols[:,1]))

['0' '1']


"Term"

In [15]:
print(np.unique(string_cols[:,2]))

['' '36 months' '60 months']


In [16]:
string_cols[:,2] = np.char.replace(string_cols[:,2],  ' months',  '')
print(np.unique(string_cols[:,2]))

['' '36' '60']


Redenumesc coloana, ca se fie clar ca acestea sun luni:

In [17]:
string_header[2]="term_months"

In loc de datele lipsa, setez 60 luni (worst case):

In [18]:
string_cols[:,2] = np.where(string_cols[:,2]=="", "60", string_cols[:,2])
print(np.unique(string_cols[:,2]))

['36' '60']


"Grade" , "Subgrade"= clasa de risc a unui debitor

In [19]:
print(np.unique(string_cols[:,3]))
print(np.unique(string_cols[:,4]))

['' 'A' 'B' 'C' 'D' 'E' 'F' 'G']
['' 'A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5' 'D1' 'D2' 'D3' 'D4' 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']


In [20]:
string_cols[:,4]=np.where((string_cols[:,4]=="") & (string_cols[:,3]!=""), np.char.add(string_cols[:,3],"5"),string_cols[:,4])
print(np.unique(string_cols[:,4]))

['' 'A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5' 'D1' 'D2' 'D3' 'D4' 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']


In [21]:
string_cols[:,4]=np.where(string_cols[:,4]=="", "G5",string_cols[:,4])
print(np.unique(string_cols[:,4]))

['A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5' 'D1' 'D2' 'D3' 'D4' 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']


In [22]:
string_cols=np.delete(string_cols,3,axis=1)
string_header=np.delete(string_header,3)
print(string_cols, end="\n\n")
print(string_header)

[['5' '1' '36' ... 'Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226' 'CA']
 ['0' '1' '36' ... 'Source Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261' 'NY']
 ['9' '1' '36' ... 'Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726' 'PA']
 ...
 ['6' '1' '36' ... 'Source Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990' 'CA']
 ['4' '1' '36' ... 'Source Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151' 'OH']
 ['12' '1' '36' ... '' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249' 'IL']]

['issue_month' 'loan_status' 'term_months' 'sub_grade' 'verification_status' 'url' 'addr_state']


Asociez cate un numar pentru fiecare subgrade:

In [23]:
subgrade_keys=np.unique(string_cols[:,3])
subgrade_values=range(1,np.unique(string_cols[:,3]).shape[0]+1)
subgrade_dict=dict(zip(subgrade_keys,subgrade_values))
print(subgrade_dict)

{np.str_('A1'): 1, np.str_('A2'): 2, np.str_('A3'): 3, np.str_('A4'): 4, np.str_('A5'): 5, np.str_('B1'): 6, np.str_('B2'): 7, np.str_('B3'): 8, np.str_('B4'): 9, np.str_('B5'): 10, np.str_('C1'): 11, np.str_('C2'): 12, np.str_('C3'): 13, np.str_('C4'): 14, np.str_('C5'): 15, np.str_('D1'): 16, np.str_('D2'): 17, np.str_('D3'): 18, np.str_('D4'): 19, np.str_('D5'): 20, np.str_('E1'): 21, np.str_('E2'): 22, np.str_('E3'): 23, np.str_('E4'): 24, np.str_('E5'): 25, np.str_('F1'): 26, np.str_('F2'): 27, np.str_('F3'): 28, np.str_('F4'): 29, np.str_('F5'): 30, np.str_('G1'): 31, np.str_('G2'): 32, np.str_('G3'): 33, np.str_('G4'): 34, np.str_('G5'): 35}


In [24]:
string_cols[:,3]=[str(subgrade_dict[key]) for key in string_cols[:,3]]

In [25]:
print(np.unique(string_cols[:,3]))
print(string_cols[:,3])

['1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '3' '30' '31' '32' '33' '34' '35' '4' '5' '6' '7' '8' '9']
['13' '5' '10' ... '5' '17' '4']


"Verification Status"

In [26]:
print(np.unique(string_cols[:,4]))

['' 'Not Verified' 'Source Verified' 'Verified']


In [27]:
bad_status=np.array(["", "Not Verified"])
string_cols[:,4]=np.where(np.isin(string_cols[:,4], bad_status), "0", "1")
print(np.unique(string_cols[:,4]))

['0' '1']


"URL"

In [28]:
print(np.unique(string_cols[:,5]))

['https://www.lendingclub.com/browse/loanDetail.action?loan_id=12606806' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=13026045' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=1312426' ... 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=8138291'
 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=8214572' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=849994']


Difera doar ID-urile, restul pot sterge:

In [29]:
string_cols[:,5]=np.char.replace(string_cols[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=","")


In [30]:
print(np.unique(string_cols[:,5], return_counts=True))

(array(['12606806', '13026045', '1312426', ..., '8138291', '8214572', '849994'], shape=(10000,), dtype='<U69'), array([1, 1, 1, ..., 1, 1, 1], shape=(10000,)))


In [31]:
np.array_equal(string_cols[:,5].astype(np.int64), numeric_cols[:,0].astype(np.int64))

True

Avand in vedere ca coloana "URL" contine aceeasi info ca si coloana "ID", voi sterge coloana "URL"

In [32]:
string_cols=np.delete(string_cols, 5, axis=1)
string_header=np.delete(string_header, 5)
print(string_header)
print(string_cols, end="\n\n")
print(numeric_header)
print(numeric_cols)

['issue_month' 'loan_status' 'term_months' 'sub_grade' 'verification_status' 'addr_state']
[['5' '1' '36' '13' '1' 'CA']
 ['0' '1' '36' '5' '1' 'NY']
 ['9' '1' '36' '10' '1' 'PA']
 ...
 ['6' '1' '36' '5' '1' 'CA']
 ['4' '1' '36' '17' '1' 'OH']
 ['12' '1' '36' '4' '0' 'IL']]

['id' 'loan_amount' 'funded_amount' 'int_rate' 'installment' 'total_payment']
[[48010226.      35000.      35000.         13.33     1184.86     9452.96]
 [57693261.      30000.      30000.           nan      938.57     4679.7 ]
 [59432726.      15000.      15000.           nan      494.86     1969.83]
 ...
 [50415990.      10000.      10000.           nan         nan     2185.64]
 [46154151.           nan    10000.         16.55      354.3      3199.4 ]
 [66055249.      10000.      10000.           nan      309.97      301.9 ]]


"State Address"

In [33]:
string_header[5]="state_address"
print(string_header)
print(np.unique(string_cols[:,5]))

['issue_month' 'loan_status' 'term_months' 'sub_grade' 'verification_status' 'state_address']
['' 'AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD' 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY']


Impart statele pe zone geografice:

In [34]:
west_states=np.array(["WA", "OR", "CA", "NV", "ID", "MT", "WY", "UT", "CO", "AZ", "NM", "HI", "AK"])
east_states=np.array(["PA", "NY", "NJ", "CT", "MA", "VT", "NH", "ME", "RI"])
south_states=np.array(["TX", "OK","AR", "LA", "MS", "AL","TN", "KY","FL", "GA", "SC","NC","VA","WV","MD","DE","DC"])
midwest_states=np.array(["ND", "SD", "NE", "KS", "MN", "IA", "MO", "WI", "IL", "IN", "MI", "OH"])

In [35]:
string_cols[:,5]=np.where(np.isin(string_cols[:,5], west_states), "1", string_cols[:,5])
string_cols[:,5]=np.where(np.isin(string_cols[:,5], south_states), "2", string_cols[:,5])
string_cols[:,5]=np.where(np.isin(string_cols[:,5], midwest_states), "3", string_cols[:,5])
string_cols[:,5]=np.where(np.isin(string_cols[:,5], east_states), "4", string_cols[:,5])
string_cols[:,5]=np.where(string_cols[:,5]=="", "0", string_cols[:,5])

In [36]:
print(np.unique(string_cols[:,5]))
print(string_cols[:,5])

['0' '1' '2' '3' '4']
['1' '4' '4' ... '1' '3' '3']


Convertesc string_cols in int:

In [37]:
print(string_cols)

[['5' '1' '36' '13' '1' '1']
 ['0' '1' '36' '5' '1' '4']
 ['9' '1' '36' '10' '1' '4']
 ...
 ['6' '1' '36' '5' '1' '1']
 ['4' '1' '36' '17' '1' '3']
 ['12' '1' '36' '4' '0' '3']]


In [38]:
string_cols=string_cols.astype(np.int64)
print(string_cols)

[[ 5  1 36 13  1  1]
 [ 0  1 36  5  1  4]
 [ 9  1 36 10  1  4]
 ...
 [ 6  1 36  5  1  1]
 [ 4  1 36 17  1  3]
 [12  1 36  4  0  3]]


Datele numerice:

In [39]:
print(numeric_header)
print(numeric_cols)

['id' 'loan_amount' 'funded_amount' 'int_rate' 'installment' 'total_payment']
[[48010226.      35000.      35000.         13.33     1184.86     9452.96]
 [57693261.      30000.      30000.           nan      938.57     4679.7 ]
 [59432726.      15000.      15000.           nan      494.86     1969.83]
 ...
 [50415990.      10000.      10000.           nan         nan     2185.64]
 [46154151.           nan    10000.         16.55      354.3      3199.4 ]
 [66055249.      10000.      10000.           nan      309.97      301.9 ]]


Verific care coloane contin valori nan:

In [40]:
np.all(np.isnan(numeric_cols[:,0])==False)

np.True_

"loan amount" = suma pe care debitorul (persoana care ia creditul) a solicitat-o
"funded amount" = suma efectiva care a fost finantata de creditor
"interest rate" = rata de dobanda a bancii
"installment" = rata lunara pe care trebuie sa o plateasca debitorul
"total payment" = suma totala pe care o plateste debitorul (suma tuturor ratelor)

In [41]:
numeric_header[3]="interest_rate"
print(numeric_header)

['id' 'loan_amount' 'funded_amount' 'interest_rate' 'installment' 'total_payment']


Valori worst case (in loc de nan): loan_amount->max, funded_amount->min, interest_rate->max, installment->max, total_payment->max

In [42]:
numeric_cols[:,1]=np.where(np.isnan(numeric_cols[:,1]), np.nanmax(numeric_cols[:,1]), numeric_cols[:,1])
numeric_cols[:,2]=np.where(np.isnan(numeric_cols[:,2]), np.nanmin(numeric_cols[:,2]), numeric_cols[:,2])
numeric_cols[:,3]=np.where(np.isnan(numeric_cols[:,3]), np.nanmax(numeric_cols[:,3]), numeric_cols[:,3])
numeric_cols[:,4]=np.where(np.isnan(numeric_cols[:,4]), np.nanmax(numeric_cols[:,4]), numeric_cols[:,4])
numeric_cols[:,5]=np.where(np.isnan(numeric_cols[:,5]), np.nanmax(numeric_cols[:,5]), numeric_cols[:,5])

In [43]:
print(numeric_cols)

[[48010226.      35000.      35000.         13.33     1184.86     9452.96]
 [57693261.      30000.      30000.         28.99      938.57     4679.7 ]
 [59432726.      15000.      15000.         28.99      494.86     1969.83]
 ...
 [50415990.      10000.      10000.         28.99     1372.97     2185.64]
 [46154151.      35000.      10000.         16.55      354.3      3199.4 ]
 [66055249.      10000.      10000.         28.99      309.97      301.9 ]]


Unesc cele 2 array-uri (string si numeric):

In [44]:
print(numeric_cols.shape)
print(string_cols.shape)

(10000, 6)
(10000, 6)


In [45]:
preprocessed_data=np.hstack((numeric_cols, string_cols))
full_header=np.hstack((numeric_header, string_header))
print(preprocessed_data.shape)
print(full_header)
print(preprocessed_data)

(10000, 12)
['id' 'loan_amount' 'funded_amount' 'interest_rate' 'installment' 'total_payment' 'issue_month' 'loan_status' 'term_months' 'sub_grade' 'verification_status' 'state_address']
[[48010226.    35000.    35000. ...       13.        1.        1.]
 [57693261.    30000.    30000. ...        5.        1.        4.]
 [59432726.    15000.    15000. ...       10.        1.        4.]
 ...
 [50415990.    10000.    10000. ...        5.        1.        1.]
 [46154151.    35000.    10000. ...       17.        1.        3.]
 [66055249.    10000.    10000. ...        4.        0.        3.]]


In [46]:
np.all(np.isnan(preprocessed_data)==False)

np.True_

In [47]:
preprocessed_data=preprocessed_data[np.argsort(preprocessed_data[:,0])]
print(preprocessed_data)

[[  373332.     9950.     1000. ...       21.        0.        1.]
 [  575239.    12000.    12000. ...       25.        1.        2.]
 [  707689.    10000.    10000. ...       13.        1.        0.]
 ...
 [68614880.     5600.     5600. ...        8.        1.        1.]
 [68615915.     4000.     4000. ...       10.        1.        2.]
 [68616519.    21600.    21600. ...        3.        0.        2.]]


Verific daca array-ul e corect sortat:

In [48]:
np.argsort(preprocessed_data[:,0])

array([   0,    1,    2, ..., 9997, 9998, 9999], shape=(10000,))

Concatenez cele 2 array-uri:full_header si preprocessed_data

In [49]:
full_data=np.vstack((full_header, preprocessed_data))
print(full_data)

[['id' 'loan_amount' 'funded_amount' ... 'sub_grade' 'verification_status' 'state_address']
 ['373332.0' '9950.0' '1000.0' ... '21.0' '0.0' '1.0']
 ['575239.0' '12000.0' '12000.0' ... '25.0' '1.0' '2.0']
 ...
 ['68614880.0' '5600.0' '5600.0' ... '8.0' '1.0' '1.0']
 ['68615915.0' '4000.0' '4000.0' ... '10.0' '1.0' '2.0']
 ['68616519.0' '21600.0' '21600.0' ... '3.0' '0.0' '2.0']]


In [50]:
np.savetxt("Preprocessed_data.csv", full_data, "%s", delimiter=";")